In [3]:
import numpy as np
from enum import Enum

In [6]:
class Direction(Enum):
    NONE = 0
    LEFT = 1
    UP = 2
    RIGHT = 3
    DOWN = 4

In [ ]:
def none():
    return 
def left():
    print "n is a perfect square\n"
def up():
    print "n is an even number\n"
def right():
    print "n is a prime number\n"
def down():
    print "n is a prime number\n"

In [ ]:
# Our partial state (without the minotaur)
class PartialState:
    
    def __init__(x, y):
        self.x = x
        self.y = y
        
    def getActions():
        # TODO: fetch all possible actions
        return [Direction.NONE, Direction.LEFT, Direction.UP, Direction.RIGHT, Direction.DOWN]
    
    def act(direction):
        # TODO: act or through an exception
        ##if direction is Direction.NONE:
            
        
    

In [5]:
map_height=5
map_width=6
position = (0,1)
walls = { (1,0): [Direction.RIGHT], (1,1): [Direction.RIGHT], (1,2): [Direction.RIGHT],
        (3,1): [Direction.RIGHT], (3,2): [Direction.RIGHT],
        (4,1): [Direction.DOWN], (5,1): [Direction.DOWN],
        (1,3): [Direction.DOWN], (2,3): [Direction.DOWN], (3,3): [Direction.DOWN], (4,3): [Direction.DOWN],
        (3,4): [Direction.LEFT] }

In [ ]:
actions = {(3,1):[]}